In [1]:
# Import Dependencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC

# DATA PREPARATION

In [2]:
# Load Malaria Dataset From Tensorflow Datasets
dataset, datasetInfo = tfds.load("malaria", 
                                 with_info=True, 
                                 shuffle_files=True) # -----------++++++    ->    ---+++-++--+++-

2025-01-22 17:20:48.541393: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-22 17:20:48.541418: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-22 17:20:48.541423: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-22 17:20:48.541437: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-22 17:20:48.541448: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Dataset Split Function
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
    trainDataset = dataset.take(int(TRAIN_RATIO*len(dataset)))
    leftDataset = dataset.skip(int(TRAIN_RATIO*len(dataset)))
    valDataset = leftDataset.take(int(VAL_RATIO*len(dataset)))
    testDataset = leftDataset.skip(int(VAL_RATIO*len(dataset)))
    return trainDataset, valDataset, testDataset

In [4]:
# Split Dataset As 80/10/10 
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
trainDataset, valDataset, testDataset = split(dataset['train'], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

In [5]:
IMAGE_SIZE = 224
def resizeRescale(inputs):
    return tf.image.resize(inputs['image'], (IMAGE_SIZE, IMAGE_SIZE))/255., inputs['label']
trainDataset = trainDataset.map(resizeRescale)
valDataset = valDataset.map(resizeRescale)
testDataset = testDataset.map(resizeRescale)

In [8]:
# Function To Extract Labels From Output
def getLabel(label):
    if label==0:
        return "Parasitized"
    return "Unparasitized"

# DATA MODELING -> Convolutional Neural Network

In [9]:
# Using Sequential API
IMAGE_SIZE = 224
model = tf.keras.Sequential([
    Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),

    Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [10]:
# Model Metrics
Metrics = [BinaryAccuracy(), FalsePositives(), FalseNegatives(), TrueNegatives(), TruePositives(), Precision(), Recall(), AUC()]